# Predictive Analytics for Credit Default

**Objective:** Build a clean, reproducible notebook that performs Exploratory Data Analysis (EDA), Feature Engineering, and Model Training for predicting credit-card default. This notebook is formatted for submission to an MSc Data Science professor.

**Notes for reviewer:**
- The dataset should be placed locally and the path updated in the `DATA_PATH` variable.
- The notebook focuses only on EDA, feature engineering, and model training with clear, reproducible steps.


## 1. Setup and Imports

Install any missing packages before running (run in a terminal or notebook cell):
```
pip install pandas numpy matplotlib seaborn scikit-learn lightgbm joblib
```


In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.metrics import (
    roc_auc_score, average_precision_score, roc_curve, precision_recall_curve,
    classification_report, confusion_matrix, brier_score_loss
)
import joblib
from scipy.stats import randint, uniform

sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (9,6)

RND = 42
os.makedirs('models', exist_ok=True)


## 2. Data Loading & Quick Checks

Update `DATA_PATH` to point at your local CSV. The notebook assumes the UCI Credit Card dataset with column `default.payment.next.month`.


In [ ]:
# Path to your CSV - update if needed
DATA_PATH = r"D:\Data Science\financial-risk-predictive-analytics\UCI_Credit_Card.csv"

assert os.path.exists(DATA_PATH), f"Data file not found at {DATA_PATH}. Please update DATA_PATH."

df = pd.read_csv(DATA_PATH)
df = df.rename(columns={"default.payment.next.month": "default"})
if 'ID' in df.columns:
    df = df.drop(columns=['ID'])

print('Shape:', df.shape)
display(df.head())
print('\nColumn dtypes:')
display(df.dtypes.value_counts())


### Data quality checks
- Look for missing values
    - Dataset should not contain missing values by design; if there are any, we handle them conservatively later.


In [ ]:
missing = df.isnull().sum()
missing[missing>0]


## 3. Target distribution and class balance
Understanding class balance is important for metric selection and modeling choices.


In [ ]:
target_counts = df['default'].value_counts().sort_index()
target_pct = df['default'].value_counts(normalize=True).sort_index() * 100
display(pd.DataFrame({'count': target_counts, 'percentage': target_pct.round(2)}))

sns.countplot(x='default', data=df)
plt.title('Target Distribution (Non-default=0 vs Default=1)')
plt.show()


## 4. Categorical value validation
Some categorical codes in the original dataset include undocumented values (e.g., EDUCATION=0,5,6 or MARRIAGE=0). Consolidate them into a single 'Other' code for stability.


In [ ]:
# Consolidate undocumented categories
df['EDUCATION'] = df['EDUCATION'].replace({0:4, 5:4, 6:4})
df['MARRIAGE'] = df['MARRIAGE'].replace({0:3})

display(df[['EDUCATION','MARRIAGE']].apply(pd.Series.value_counts))


## 5. Distribution inspection (key numeric features)
Plot distributions for `LIMIT_BAL` and `AGE` to inspect skewness and outliers.


In [ ]:
def plot_distribution(col, bins=40):
    sns.histplot(df[col], bins=bins, kde=True, stat='density')
    plt.title(f"{col} | skew={df[col].skew():.2f}")
    plt.xlabel(col)
    plt.ylabel('Density')
    plt.show()

for col in ['LIMIT_BAL','AGE']:
    plot_distribution(col)


## 6. Delinquency features & business-focused engineered variables
Create aggregated features that are interpretable and predictive:
- `num_delayed_payments`: count of months with PAY_*>0
- `ever_late`: binary flag if customer was late in any month
- `total_bill_amt`, `total_pay_amt`
- `payment_ratio` and `credit_utilization`


In [ ]:
pay_cols = [c for c in df.columns if c.startswith('PAY_') and c not in ('PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6')]
bill_cols = [c for c in df.columns if c.startswith('BILL_AMT')]
pay_amt_cols = [c for c in df.columns if c.startswith('PAY_AMT')]

df['num_delayed_payments'] = (df[pay_cols] > 0).sum(axis=1)
df['ever_late'] = (df[pay_cols] > 0).any(axis=1).astype(int)

df['total_bill_amt'] = df[bill_cols].sum(axis=1)
df['total_pay_amt'] = df[pay_amt_cols].sum(axis=1)

df['payment_ratio'] = df['total_pay_amt'] / (df['total_bill_amt'] + 1)
df['credit_utilization'] = df['total_bill_amt'] / (df['LIMIT_BAL'] + 1)

sns.boxplot(x='default', y='num_delayed_payments', data=df)
plt.title('Number of Delayed Payments vs Default')
plt.show()

sns.boxplot(x='default', y='payment_ratio', data=df)
plt.ylim(0,3)
plt.title('Payment Ratio vs Default')
plt.show()


## 7. Recent behavior features
Recent months are typically more predictive; compute mean of last 3 months for bills/payments.


In [ ]:
df['recent_bill_mean'] = df[['BILL_AMT1','BILL_AMT2','BILL_AMT3']].mean(axis=1)
df['recent_pay_mean'] = df[['PAY_AMT1','PAY_AMT2','PAY_AMT3']].mean(axis=1)

display(df[['recent_bill_mean','recent_pay_mean']].describe())


## 8. Correlation analysis (heatmap)
Inspect correlations to find strong linear relationships and potential multicollinearity (helpful for linear models and interpretation).


In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df.corr(), cmap='coolwarm', center=0)
plt.title('Feature Correlation Heatmap')
plt.show()


## 9. Feature relevance estimation
Estimate linear (point-biserial) and non-linear (mutual information) relationships with target.


In [ ]:
# Point-biserial correlation
num_features = df.select_dtypes(include=[np.number]).columns.drop('default')
pb = {col: stats.pointbiserialr(df['default'], df[col]).correlation for col in num_features}
pb = pd.Series(pb).sort_values(key=lambda x: x.abs(), ascending=False)
display(pb.head(20))

# Mutual information (needs encoding for categorical variables)
df_mi = df.copy()
for c in df_mi.select_dtypes(include=['object','category']).columns:
    df_mi[c] = LabelEncoder().fit_transform(df_mi[c].astype(str))

mi = pd.Series(
    mutual_info_classif(df_mi.drop(columns=['default']), df_mi['default'], random_state=RND),
    index=df_mi.drop(columns=['default']).columns
).sort_values(ascending=False)
display(mi.head(20))


## 10. Final feature selection for modeling
Build a compact, interpretable feature set suitable for API deployment and model auditing.


In [ ]:
selected_features = [
    'LIMIT_BAL', 'total_bill_amt', 'total_pay_amt', 'payment_ratio',
    'credit_utilization', 'num_delayed_payments', 'ever_late',
    'AGE', 'SEX', 'EDUCATION', 'MARRIAGE', 'recent_bill_mean'
]
final_features = [f for f in selected_features if f in df.columns]
missing_feats = set(selected_features) - set(final_features)
if missing_feats:
    print('Warning: missing features will be excluded:', missing_feats)

final_df = df[final_features].copy()

# Conservative imputation and dtype normalization
num_cols = final_df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = final_df.select_dtypes(include=['object','category']).columns.tolist()

for c in num_cols:
    final_df[c] = final_df[c].fillna(final_df[c].median())
for c in cat_cols:
    if final_df[c].isnull().any():
        final_df[c] = final_df[c].fillna(final_df[c].mode().iloc[0])

for c in ['SEX','EDUCATION','MARRIAGE']:
    if c in final_df.columns:
        final_df[c] = final_df[c].astype('category')

print('final_df shape:', final_df.shape)
display(final_df.head())


## 11. Train/Test split
Perform a stratified split to preserve class balance between train and test sets.


In [ ]:
X = final_df.copy()
y = df['default'].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=RND
)
print('Train:', X_train.shape, 'Test:', X_test.shape)


## 12. Preprocessing pipelines & Models
Construct numeric and categorical transformers and define three model pipelines: Logistic Regression, Random Forest, LightGBM.


In [ ]:
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
cat_features = X.select_dtypes(include=['category','object']).columns.tolist()

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', cat_transformer, cat_features)
], remainder='drop')

logreg_pipe = Pipeline([('pre', preprocessor), ('clf', LogisticRegression(solver='saga', max_iter=5000, random_state=RND))])
rf_pipe = Pipeline([('pre', preprocessor), ('clf', RandomForestClassifier(n_jobs=-1, random_state=RND))])
lgb_pipe = Pipeline([('pre', preprocessor), ('clf', lgb.LGBMClassifier(random_state=RND, n_jobs=-1))])


## 13. Hyperparameter search setup (RandomizedSearchCV)
Use `roc_auc` as the primary scoring metric. `n_iter` kept moderate for speed in a classroom setting.


In [ ]:
param_dist = {
    'logreg': {
        'clf__penalty': ['l2','l1'],
        'clf__C': uniform(loc=0.01, scale=10.0),
        'clf__class_weight': [None, 'balanced']
    },
    'rf': {
        'clf__n_estimators': randint(100, 400),
        'clf__max_depth': [None, 5, 10, 20],
        'clf__min_samples_split': randint(2, 8),
        'clf__max_features': ['sqrt','log2', 0.5, None],
        'clf__class_weight': [None, 'balanced', 'balanced_subsample']
    },
    'lgb': {
        'clf__n_estimators': randint(100, 400),
        'clf__num_leaves': randint(16, 128),
        'clf__learning_rate': uniform(loc=0.01, scale=0.3),
        'clf__max_depth': [-1, 3, 6, 12],
        'clf__subsample': uniform(loc=0.6, scale=0.4)
    }
}

cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=RND)
n_iter = 24


In [ ]:
def tune_model(pipe, params, X, y, cv, n_iter=24, scoring='roc_auc'):
    rs = RandomizedSearchCV(pipe, params, n_iter=n_iter, cv=cv, scoring=scoring,
                            random_state=RND, n_jobs=-1, verbose=1, return_train_score=False)
    rs.fit(X, y)
    print('Best score (cv):', rs.best_score_)
    print('Best params:', rs.best_params_)
    return rs


### 13.1 Tune and save models
We run a shorter search for classroom speed. In a production or thesis run you may increase `n_iter` and `cv` folds.


In [ ]:
print('Tuning Logistic Regression...')
rs_log = tune_model(logreg_pipe, param_dist['logreg'], X_train, y_train, cv=cv, n_iter=20)
best_log = rs_log.best_estimator_
joblib.dump(best_log, 'models/logreg_pipeline.joblib')

print('\nTuning Random Forest...')
rs_rf = tune_model(rf_pipe, param_dist['rf'], X_train, y_train, cv=cv, n_iter=24)
best_rf = rs_rf.best_estimator_
joblib.dump(best_rf, 'models/rf_pipeline.joblib')

print('\nTuning LightGBM...')
scale_pos_weight = (y_train==0).sum() / (y_train==1).sum()
param_dist_lgb = param_dist['lgb'].copy()
param_dist_lgb['clf__scale_pos_weight'] = [scale_pos_weight, None]
rs_lgb = tune_model(lgb_pipe, param_dist_lgb, X_train, y_train, cv=cv, n_iter=24)
best_lgb = rs_lgb.best_estimator_
joblib.dump(best_lgb, 'models/lgbm_pipeline.joblib')


## 14. Evaluation on hold-out test set
Compute ROC-AUC, PR-AUC, classification report, confusion matrix, and plot ROC & PR curves.


In [ ]:
models = {'LogisticRegression': best_log, 'RandomForest': best_rf, 'LightGBM': best_lgb}
results = {}

plt.figure(figsize=(8,6))
for name, model in models.items():
    y_prob = model.predict_proba(X_test)[:,1]
    y_pred = model.predict(X_test)
    roc = roc_auc_score(y_test, y_prob)
    pr = average_precision_score(y_test, y_prob)
    results[name] = {'roc_auc': float(roc), 'pr_auc': float(pr)}
    print(f"--- {name} ---")
    print('ROC-AUC:', roc)
    print('PR-AUC:', pr)
    print(classification_report(y_test, y_pred, digits=4))
    print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    plt.plot(fpr, tpr, label=f"{name} (AUC={roc:.3f})")

plt.plot([0,1],[0,1],'k--', alpha=0.4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - Test Set')
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for name, model in models.items():
    y_prob = model.predict_proba(X_test)[:,1]
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    ap = average_precision_score(y_test, y_prob)
    plt.plot(recall, precision, label=f"{name} (AP={ap:.3f})")
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves - Test Set')
plt.legend()
plt.show()


In [ ]:
# Calibration plots
from sklearn.calibration import calibration_curve
plt.figure(figsize=(12,4))
for i, (name, model) in enumerate(models.items(), 1):
    y_prob = model.predict_proba(X_test)[:,1]
    prob_true, prob_pred = calibration_curve(y_test, y_prob, n_bins=10)
    plt.subplot(1,3,i)
    plt.plot(prob_pred, prob_true, marker='o', label=name)
    plt.plot([0,1],[0,1],'k--', alpha=0.4)
    plt.xlabel('Mean predicted prob')
    plt.ylabel('Fraction of positives')
    plt.title(f"{name} (Brier={brier_score_loss(y_test, y_prob):.4f})")
    plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Save results summary
with open('models/results_summary.json','w') as f:
    json.dump(results, f, indent=2)
print('Saved models and results in ./models/')


## 15. Conclusion & next steps (for thesis / production)
- Random forest / LightGBM often outperform simple logistic regression on this dataset (check ROC/PR metrics above).
- Next steps: feature importance analysis, SHAP explanations for model interpretability, stronger hyperparameter tuning, cross-validation across multiple random seeds, and productionization (model card, API, monitoring).
- Ensure reproducibility: pin package versions and include a `requirements.txt` in your GitHub repo.
